# Srovnání SVD a randomizovaného SVD

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [76]:
n = 1001
k = 1000
X_dense = np.random.rand(n**2)
X_dense[np.random.rand(n**2) < 0.9] = 0
X_dense = X_dense.reshape(n, n)
X = csr_matrix(X_dense)
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 99759 stored elements and shape (1001, 1001)>
  Coords	Values
  (0, 11)	0.34519477414654165
  (0, 21)	0.6565087684317773
  (0, 23)	0.3700857739553558
  (0, 28)	0.5668255022543808
  (0, 54)	0.5374438373238114
  (0, 55)	0.9573282339683717
  (0, 67)	0.7907310265854844
  (0, 101)	0.6426645612120442
  (0, 128)	0.7572162524910092
  (0, 131)	0.8632768063456431
  (0, 136)	0.7761949543152885
  (0, 146)	0.1744559734381529
  (0, 150)	0.2602285092491531
  (0, 156)	0.9434733905054608
  (0, 165)	0.595064314880093
  (0, 166)	0.510234310385833
  (0, 178)	0.40845300444696553
  (0, 184)	0.2397925400614671
  (0, 207)	0.8881934955358104
  (0, 224)	0.6986804478635499
  (0, 230)	0.9587332198469389
  (0, 244)	0.7250607651787607
  (0, 272)	0.5374107531202705
  (0, 276)	0.004080114593359108
  (0, 296)	0.9330385800656991
  :	:
  (1000, 718)	0.6664922062830749
  (1000, 722)	0.4131275644625987
  (1000, 725)	0.7150344887916902
  (1000, 730)	0.3246306861

In [77]:
%%time
for i in range(10):
    svd = TruncatedSVD(n_components=k, algorithm="arpack")
    svd.fit(X)

CPU times: total: 3min 40s
Wall time: 3min 6s


In [78]:
%%time
for i in range(10):
    rsvd = TruncatedSVD(n_components=k, algorithm="randomized")
    rsvd.fit(X)

CPU times: total: 2min 31s
Wall time: 38.4 s
